# Определение стоимости автомобилей

Нам нужно построить модель для определения рыночной стоимости автомобиля.\
Шаги:\
1)Загрузим данные.\
2)Изучим данные. Заполним пропущенные значения и обработаем аномалии в столбцах.\
3)Подготовим выборки для обучения моделей.\
4)Обучим разные модели. \
5)Проанализируем качество моделей.\
6)Выберем лучшую модель, проверим её качество на тестовой выборке.

## 1. Загрузка данных

Импортируем все необходимые для работы библиотеки.

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import re


Загрузим наши данные и проведем первичный анализ.

In [2]:
data = pd.read_csv('autos.csv')
display(data.head(5))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


# 2. Исследование данных

In [3]:
# Преобразование названий столбцов в snake_case
data.columns = [''.join(['_'+ c.lower() if c.isupper() else c for c in col]).lstrip('_') for col in data.columns]
data.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

В наших данных целевая переменая - price(цена автомобиля). Как мы можем заметить, имеются пропуски в VehicleType(тип автомобиля), Model(модель автомобиля), и в признаке Repaired("битый" автомобиль).
Кроме того, для признака power(мощность автомобиля) есть странное значение 0, у автомобиля не может быть нулевой мощности.
Исследуем наши данные подробнее на пропуски, дубликаты и выбросы(аномалии).

In [4]:
# Посчитаем процентное соотношение пропусков для каждого признака

data.isna().sum() / data.count() * 100

date_crawled           0.000000
price                  0.000000
vehicle_type          11.831014
registration_year      0.000000
gearbox                5.928510
power                  0.000000
model                  5.887995
kilometer              0.000000
registration_month     0.000000
fuel_type             10.232554
brand                  0.000000
repaired              25.123669
date_created           0.000000
number_of_pictures     0.000000
postal_code            0.000000
last_seen              0.000000
dtype: float64

В наших данных много пропусков (vehicle_type, gearbox, model, fuel_type, repaired), чтобы не потерять информативности заполним их. Каждый рассмотрим подробнее.

In [5]:
#Рассмотрим признакми vehicle_type и model

display(data['vehicle_type'].value_counts())
display(data['model'].value_counts())

vehicle_type
sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: count, dtype: int64

model
golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: count, Length: 250, dtype: int64

Как мы можем заметить, разница между первыми 3 позициями не сильно большая, а значит заполнить пропуски наиболее популярными будет неверным решением. Для обоих признаков пропуски неочевидны. Тогда заполним их значением заглушкой "unknown".

In [6]:
data[['model', 'vehicle_type']] = data[['model', 'vehicle_type']].fillna('unknown')

Пропуски в оставшихся признаках скорее всего подзразумевают настолько банальное значение, что пользователь решил их не заполнять\
Так, для признака gearbox - это механическая коробка передач(manual). Большинство машин на механической коробке передач.\
Для признака fuel_type - 'fueltype'. Большинство машин выпускают изначально на бензиновом топливе.\
Для признака not_repaired - no. Скорее всего, пропуск означает, что машина не попадала в аварии.

In [7]:
data['gearbox'] = data['gearbox'].fillna('manual')
data['fuel_type'] = data['fuel_type'].fillna('petrol')
data['repaired'] = data['repaired'].fillna('no')

In [8]:
print(data['registration_year'].apply(['max','min']), end='\n\n')
print(data['power'].apply(['max','min']))
print(data['price'].apply(['max','min']))

max    9999
min    1000
Name: registration_year, dtype: int64

max    20000
min        0
Name: power, dtype: int64
max    20000
min        0
Name: price, dtype: int64


Как мы можем заметить в столбцах power и registration_year есть аномалии избавимся от них. Год регистрации ограничим снизу 1960 годом а сверзу 2023.Мощность снизу 50 а сверху 2028(максимальная мощность машины на данный момент).\
Цену ограничим снизу 300( цена указана в евро и примерно равна 30000, машины дешевле данной суммы можно отнести к металлалому)

In [9]:
#Удалим выбросы
data = data.query('registration_year < 2017 and registration_year > 1960')
data = data.query('power > 50 and power < 2028')
data = data.query('price > 350')

In [10]:
#Посмотрим сколько осталось данных
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276742 entries, 1 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        276742 non-null  object
 1   price               276742 non-null  int64 
 2   vehicle_type        276742 non-null  object
 3   registration_year   276742 non-null  int64 
 4   gearbox             276742 non-null  object
 5   power               276742 non-null  int64 
 6   model               276742 non-null  object
 7   kilometer           276742 non-null  int64 
 8   registration_month  276742 non-null  int64 
 9   fuel_type           276742 non-null  object
 10  brand               276742 non-null  object
 11  repaired            276742 non-null  object
 12  date_created        276742 non-null  object
 13  number_of_pictures  276742 non-null  int64 
 14  postal_code         276742 non-null  int64 
 15  last_seen           276742 non-null  object
dtypes: int6

Удалим неинформативные признаки: datacrawled,datecreated,lastseen,number_of_pictures,registretion_month скорее всего цена от них не зависит.\
Разделим данные на features и target.

In [11]:
features = data.drop(['price','date_crawled','date_created','last_seen','postal_code','number_of_pictures','registration_month'],axis=1)
target = data['price']

# 3. Подготовим наши выборки для обучения

Перед обучением разобьем данные на обучающую, валидационную и тестовую выборки.

In [12]:
features_train, features_valid,target_train,target_valid = train_test_split(features,target,test_size=0.4,random_state=12345)
features_valid,features_test,target_valid,target_test = train_test_split(features_valid,target_valid,test_size = 0.5,random_state=12345)

In [13]:
# Посмотрим на наши данные
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166045 entries, 273345 to 278467
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   vehicle_type       166045 non-null  object
 1   registration_year  166045 non-null  int64 
 2   gearbox            166045 non-null  object
 3   power              166045 non-null  int64 
 4   model              166045 non-null  object
 5   kilometer          166045 non-null  int64 
 6   fuel_type          166045 non-null  object
 7   brand              166045 non-null  object
 8   repaired           166045 non-null  object
dtypes: int64(3), object(6)
memory usage: 12.7+ MB


Так как мы будем использовать линейную регрессию нам нужно закодировать категориальные данные и отмасштабировать целевые признаки.

In [14]:
# Закодируем наши категориальные данные

features_train_ohe = pd.get_dummies(features_train, drop_first=True).astype('int')
features_valid_ohe = pd.get_dummies(features_valid, drop_first=True).astype('int')
features_test_ohe = pd.get_dummies(features_test, drop_first=True).astype('int')

Теперь нужно проверить, что признаки в каждой выборке совпадают.

In [15]:
# Посмотрим на количество признаков
features_train_ohe.shape, features_valid_ohe.shape, features_test_ohe.shape

((166045, 306), (55348, 302), (55349, 300))

Как мы видим в данных некоторые признаки отличаются, избавимся от них. Напишем функцию, осуществляющую проверку: если какого-то признака нет, то мы его удаляем 

In [16]:
def same_features(features_one,features_second):
    features_one.astype('int')
    for col in features_one.columns.to_list():
        if col not in features_second.columns.to_list():
            del features_one[col]

Применим нашу функцию к выборкам.

In [17]:
features_list = [features_train_ohe, features_valid_ohe, features_test_ohe]
for el in features_list:
    for el_sec in features_list:
        same_features(el,el_sec)

Теперь проверим наши признаки.

In [18]:
features_train_ohe.shape, features_valid_ohe.shape, features_test_ohe.shape

((166045, 298), (55348, 298), (55349, 298))

Как мы видим, количество признаков совпрадает. Можно перейти к стандартизации

In [19]:
scaler = StandardScaler()
numeric = ['registration_year', 'power', 'kilometer']

scaler.fit(features_train_ohe[numeric])

features_train_ohe[numeric] = scaler.transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])
features_test_ohe[numeric] = scaler.transform(features_test_ohe[numeric])

Закодируем численные признаки

In [20]:
# Закодируем наши данные методом OE
encoder = OrdinalEncoder()
cat_col = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']

features_train_oe = features_train.copy()
features_valid_oe = features_valid.copy()
features_test_oe = features_test.copy()

features_train_oe[cat_col] = encoder.fit_transform(features_train_oe[cat_col])
features_valid_oe[cat_col] = encoder.transform(features_valid_oe[cat_col])
features_test_oe[cat_col] = encoder.transform(features_test_oe[cat_col])

In [21]:
features_train_oe.head()

,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
273345,8.0,2001,1.0,145,153.0,150000,6.0,10.0,0.0
126348,4.0,2005,0.0,150,94.0,150000,2.0,20.0,0.0
121408,4.0,1999,1.0,75,115.0,150000,6.0,38.0,0.0
229056,5.0,1999,1.0,75,102.0,150000,6.0,10.0,0.0
37663,0.0,2011,0.0,140,115.0,100000,2.0,38.0,0.0


Данные готовы для обучения:

features_train_ohe, features_valid_ohe, features_test_ohe для линейной регресии
    
features_train_oe, features_valid_oe, features_test_oe для бустингов и деревьев решений

# 4. Обучение моделей

## Линейная регрессия

Для начала обучим линейную регрессию без регуляризации, измерим время обучения и прдесказания.

In [22]:
# Линейная регрессия

model = LinearRegression()

Замерим время обучения

In [23]:
%%time

# Время обучения

model.fit(features_train_ohe, target_train)

CPU times: total: 11.2 s
Wall time: 3.8 s


LinearRegression()

In [24]:
%%time

# Время предсказания

pred = model.predict(features_valid_ohe)

CPU times: total: 438 ms
Wall time: 107 ms


In [25]:
print('Linear Regression RMSE:', mean_squared_error(target_valid, pred, squared=False))

Linear Regression RMSE: 2643.1365861302643


Обучим модель с L-2 регуляризацией.

In [26]:
# Ridge

ridge = Ridge()

In [27]:
%%time

# Время обучения

ridge.fit(features_train_ohe, target_train)

CPU times: total: 2.36 s
Wall time: 884 ms


Ridge()

In [28]:
%%time

# Время предсказания

predictions = ridge.predict(features_valid_ohe)

CPU times: total: 328 ms
Wall time: 80.2 ms


In [29]:
print('RMSE для ridge: ',mean_squared_error(target_valid, predictions, squared=False))

RMSE для ridge:  2642.931213478334


Обучим модель с L-1 регуляризацией.

In [30]:
# Lasso

lasso = Lasso()

In [31]:
%%time

#Время обучения модели

lasso.fit(features_train_ohe, target_train)

CPU times: total: 17.5 s
Wall time: 4.86 s


Lasso()

In [32]:
%%time

#Время предсказания

pred = lasso.predict(features_valid_ohe)

CPU times: total: 391 ms
Wall time: 108 ms


In [33]:
print('RMSE для Lasso: ', mean_squared_error(target_valid, pred, squared=False))

RMSE для Lasso:  2674.9577814358404


Таким образом, среди линейных моделей лучше всех себя показала Ridge с показателем RMSE 2642.931213478334

## LightGBM

Обучим модель LightGBM. Подберем для нее параметры с помощью GridSearchCV.

In [34]:
%%time

#инициализация модели

model = LGBMRegressor(random_state = 12345)
param_grid = {
    'n_estimators' : [100,500,1000],
    'learning_rate' : [0.1,0.01],
    'num_leaves' : [20,25,30]
}
grid_search = GridSearchCV(
    estimator = model,
    param_grid = param_grid,
    cv=5,
    scoring = 'neg_mean_squared_error',
    verbose = 1,
    n_jobs=-1
)

CPU times: total: 0 ns
Wall time: 0 ns


In [35]:
%%time

#Обучение

grid_search.fit(features_train_oe, target_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006041 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 628
[LightGBM] [Info] Number of data points in the train set: 166045, number of used features: 9
[LightGBM] [Info] Start training from score 5154.928013
CPU times: total: 17.9 s
Wall time: 4min 43s


GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=12345), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01],
                         'n_estimators': [100, 500, 1000],
                         'num_leaves': [20, 25, 30]},
             scoring='neg_mean_squared_error', verbose=1)

In [36]:
#Модель с наименьшей метрикой
model = grid_search.best_estimator_

In [37]:
%%time
model.fit(features_train_oe,target_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007886 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 628
[LightGBM] [Info] Number of data points in the train set: 166045, number of used features: 9
[LightGBM] [Info] Start training from score 5154.928013
CPU times: total: 21.3 s
Wall time: 6.29 s


LGBMRegressor(n_estimators=1000, num_leaves=30, random_state=12345)

In [38]:
%%time

#время предсказания

predictions = model.predict(features_valid_oe)

CPU times: total: 6.16 s
Wall time: 1.56 s


In [39]:
print('LGBMRegerssor RMSE :', mean_squared_error(target_valid,predictions,squared=False))

LGBMRegerssor RMSE : 1669.8511630061125


Таким образом, модель LightGBM выдала метрику, равную  1669.8511630061125, время обучения 4.63 с, а предсказания - 1.04.

# Catboost

Воспользуемся моделю CatBoost. Подберем для нее гиперпараметры с помощью grid_search

In [ ]:
model = CatBoostRegressor(random_state=12345)

In [40]:
#Гиперпараметры

grid_params = {
    'iterations' : [500,1000,1500],
}

In [41]:
#Инициализация GridSearch

grid_search = GridSearchCV(
    estimator = model,
    param_grid = grid_params,
    cv=5,
    scoring = 'neg_mean_squared_error',
    verbose = 1,
    n_jobs=-1
)

In [42]:
#Обучение GridSearch

grid_search.fit(features_train_oe,target_train) 

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006676 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 628
[LightGBM] [Info] Number of data points in the train set: 166045, number of used features: 9
[LightGBM] [Info] Start training from score 5154.928013


GridSearchCV(cv=5,
             estimator=LGBMRegressor(n_estimators=1000, num_leaves=30,
                                     random_state=12345),
             n_jobs=-1, param_grid={'iterations': [500, 1000, 1500]},
             scoring='neg_mean_squared_error', verbose=1)

In [43]:
#Инициализируем

model = grid_search.best_estimator_

In [44]:
%%time

#Обучим модель

model.fit(features_train_oe, target_train)

[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 628
[LightGBM] [Info] Number of data points in the train set: 166045, number of used features: 9
[LightGBM] [Info] Start training from score 5154.928013
CPU times: total: 20.7 s
Wall time: 5.87 s


LGBMRegressor(iterations=500, n_estimators=1000, num_leaves=30,
              random_state=12345)

In [45]:
%%time

# Предсказание

predictions = model.predict(features_valid_oe)

[LightGBM] [Warning] Unknown parameter: iterations
CPU times: total: 5.77 s
Wall time: 1.47 s


In [46]:
#Качетсво
mean_squared_error(target_valid, predictions, squared=False)

1669.8511630061125

Наилучшая модель показала значение метрики 1682, причем время обучения равно 24,5 с, а предсказания - 74мс.

## RandomForestRegressor

Мы не будем подбирать гиперпараметры для случайного леса, т.к. он обучается слишком долго. Рассмотрим значение метрики для модели с параметрами: n_estimators = 100, max_depth=15

In [47]:
# Инициализация модели
model = RandomForestRegressor(n_estimators=100,max_depth=15,random_state=12345)


In [48]:
%%time

#время обучения

model.fit(features_train_oe,target_train)

CPU times: total: 38.5 s
Wall time: 38.6 s


RandomForestRegressor(max_depth=15, random_state=12345)

In [49]:
%%time

#время предсказания

predictions = model.predict(features_valid_oe)

CPU times: total: 1.08 s
Wall time: 1.07 s


In [50]:
print('RMSE RandomForesClassifier:', mean_squared_error(target_valid,predictions,squared=False))

RMSE RandomForesClassifier: 1695.767292404521


У нас получилась модель со значение RMSE 1695.767292404521, временем обучения - 95с, предсказания - 3.16 с.

# Анализ моделей

1.Cреди линейных моделей лучше всех себя показала Ridge с показателем RMSE 2642.931213478334

2.модель LightGBM выдала метрику, равную  1669.8511630061125, время обучения 4.63 с, а предсказания - 1.04.

3.Catboost показала значение метрики 1682, причем время обучения равно 24,5 с, а предсказания - 74мс.

4.RF получилась модель со значение RMSE 1695.767292404521, временем обучения - 95с, предсказания - 3.16 с.

Таким образом, Наилучшая модель с наименьшим RMSE это LBGMRegressor.

# Тестирование лучшей модели

Проведем тестирование лучшей модели.

In [51]:
model = LGBMRegressor(iterations=1000, max_leaves=30, random_state=12345)
model.fit(features_train_oe, target_train)
predictions = model.predict(features_test_oe)
print(mean_squared_error(target_test,predictions,squared = False))

[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] num_leaves is set=31, max_leaves=30 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] num_leaves is set=31, max_leaves=30 will be ignored. Current value: num_leaves=31
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 628
[LightGBM] [Info] Number of data points in the train set: 166045, number of used features: 9
[LightGBM] [Info] Start training from score 5154.928013
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] num_leaves is set=31, max_leaves=30 will be ignored. Current value: num_leaves=31
1744.0722474091358


На тестовой выборке наша метрика получилась равной 1744.